# Code 6
- Preprocessing Improvement
- Decision Tree

## 1/ Import Libraries

In [1]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [2]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [13]:
#CodeSection3
train = pd.read_csv('/mntDrive/MyDrive/_Data Science/3 HR Analytics (Solo)/input/train.csv')
test = pd.read_csv('/mntDrive/MyDrive/_Data Science/3 HR Analytics (Solo)/input/test.csv')
train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


## 3/ Combine both Datasets (train, test)

In [4]:
#CodeSection4
all_data = [train, test]

### Check Combined Data 
- Missing Value
- Data Types

In [6]:
#CodeSection5
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             18359 non-null  int64  
 1   city                    18359 non-null  object 
 2   city_development_index  18359 non-null  float64
 3   gender                  14261 non-null  object 
 4   relevent_experience     18359 non-null  object 
 5   enrolled_university     18017 non-null  object 
 6   education_level         17902 non-null  object 
 7   major_discipline        15521 non-null  object 
 8   experience              18300 non-null  object 
 9   company_size            13580 non-null  object 
 10  company_type            13320 non-null  object 
 11  last_new_job            17992 non-null  object 
 12  training_hours          18359 non-null  int64  
 13  target                  18359 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

## 4/ Preprocessing Improvement

### 4.1/ Identify Numerical and Categorical Features

In [17]:
#CodeSection6
# Identify all Numerical and Categorical features
numerical_features = ['city_development_index', 'training_hours']
categorical_features = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline','experience','company_size','company_type','last_new_job']

### 4.2/ Outlier Strategy

In [18]:
#CodeSection7

# Write a loop to do the same
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  train[num_var] = train[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)
  test[num_var] = test[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)

### 4.3/ Import Libraries
- Missing Value Imputation - SimpleImputer
- Preprocessing - StandardScaler, OrdinalEncoder
- Pipeline - make_pipeline, make_column_transformer
- Model - Decision Tree

In [19]:
#CodeSection8

# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Import StandardScaler
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Make and Compose Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Import Decision tree
from sklearn.tree import DecisionTreeClassifier

### 4.4/ Build Pipeline

In [20]:
#CodeSection9
# Create Preprocessor Pipeline
preprocessor = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy = 'median'),
                   StandardScaler()),numerical_features),
     (make_pipeline(SimpleImputer(strategy = 'most_frequent'),
                    OrdinalEncoder()),categorical_features))

### 4.5/ Divide Data into X and y

In [21]:
#CodeSection10
X = train.drop(['target','enrollee_id'], axis =  1)
y = train['target']

## 5/ Build Model and Fit

In [22]:
#CodeSection11
# Create Model Pipeline and Initiate Model
model = make_pipeline(preprocessor,DecisionTreeClassifier(max_depth=5))

In [23]:
#CodeSection12 
# Fit Model
model.fit(X,y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

## 6/ Check Accuracy of Model on Train Data

In [24]:
#CodeSection13
# Predict on Train Data
y_pred = model.predict(X)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [25]:
#CodeSection14
# Import metrics library
from sklearn.metrics import accuracy_score

In [26]:
#CodeSection15
# get Actual "y" variables (use "y_true" as variable)
y_true = train['target']

In [27]:
#CodeSection16
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_true, y_pred):0.1%}")

 Train Accuracy : 86.9%


## 7/ Predict and Submission

### Predict on "Test Data"

In [30]:
#CodeSection17
# Get all the X Variables from the Test Dataset
X_test = test.drop(['enrollee_id'], axis =  1)

In [31]:
#CodeSection18
# Predict on X_test Data ("X_test_prep")
X_test_prep = model.predict(X_test)

### Create Submission File

In [32]:
#CodeSection19
submission = pd.DataFrame({
    'enrollee_id' : test['enrollee_id'],
    'target' : X_test_prep
})

### Export Submission File

In [33]:
#CodeSection34
submission.to_csv('/mntDrive/MyDrive/_Data Science/3 HR Analytics (Solo)/output/O6_Decission_Tree.csv', index = False)

In [ ]:
# LB Accuracy : 0.6805